<a href="https://colab.research.google.com/github/rjac-ml/Gender-Bias-Wiki/blob/add-colab-code/ModelTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!rm -r sample_data/

In [13]:
from psutil import virtual_memory
import os 

ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

cpu_count = os.cpu_count()

print("Number  of CPU {}".format(cpu_count))

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!
Number  of CPU 8


In [8]:
! cp -r /content/drive/MyDrive/pipeline/dataset ./

### Libraries

In [9]:
%%capture
!pip install datasets transformers spacy

In [5]:
import pandas as pd
import numpy as np
from datasets import load_dataset,load_from_disk
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import spacy
import re

In [15]:
#nlp = spacy.load("en_core_web_sm")

### Data Modeling
Deep Learning and Machine learning algorithms are develop to find patterns/ statistical 
distribution in the data. This mean that if there is no Bias toward the Language use in a text the model would not be able to find a pattern (even if we de-gendered the text). As the paper mention  "*In an ideal world, we would expect little difference between texts describing men, women, and people with other gender identities, aside from the use of explicitly gendered words, like pronouns or names. A machine learning model, then, would be unable to pick up on statistical differences among gender labels (i.e., gender bias), because such differences would not exist*" https://arxiv.org/abs/2005.00614

### Data Sourcing

In [16]:
df = load_from_disk("dataset")

In [17]:
df

DatasetDict({
    train: Dataset({
        features: ['features', 'target'],
        num_rows: 4845668
    })
    test: Dataset({
        features: ['features', 'target'],
        num_rows: 2076715
    })
})

In [18]:
df = df.rename_columns({"target":"label","features":"text"})

In [19]:
df = df.map(lambda line: {"text": line["text"].lower()},num_proc=cpu_count)

#0:   0%|          | 0/605709 [00:00<?, ?ex/s]

#1:   0%|          | 0/605709 [00:00<?, ?ex/s]

#2:   0%|          | 0/605709 [00:00<?, ?ex/s]

#3:   0%|          | 0/605709 [00:00<?, ?ex/s]

#4:   0%|          | 0/605708 [00:00<?, ?ex/s]

...

#3:   0%|          | 0/259589 [00:00<?, ?ex/s]

#4:   0%|          | 0/259589 [00:00<?, ?ex/s]

#5:   0%|          | 0/259589 [00:00<?, ?ex/s]

#6:   0%|          | 0/259589 [00:00<?, ?ex/s]

#7:   0%|          | 0/259589 [00:00<?, ?ex/s]

In [20]:
model_dir = "distilbert-base-uncased"

In [21]:
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSequenceClassification.from_pretrained(model_dir, num_labels=3)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.w

In [23]:
def tokenize_function(lines):
    result = tokenizer(lines["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

In [24]:
tokenized_datasets = df.map(tokenize_function, batched=True,num_proc=cpu_count)

#0:   0%|          | 0/606 [00:00<?, ?ba/s]

#1:   0%|          | 0/606 [00:00<?, ?ba/s]

#2:   0%|          | 0/606 [00:00<?, ?ba/s]

#3:   0%|          | 0/606 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (972 > 512). Running this sequence through the model will result in indexing errors


...

#5:   0%|          | 0/260 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (525 > 512). Running this sequence through the model will result in indexing errors


#6:   0%|          | 0/260 [00:00<?, ?ba/s]

#7:   0%|          | 0/260 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (729 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (640 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (839 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (621 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (789 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

In [3]:
tokenized_datasets

NameError: ignored

In [2]:
batch_size = 280
logging_steps = len(tokenized_datasets["train"]) // batch_size
#model_name = model_dir.split("/")[-1]

NameError: ignored

In [26]:
training_args = TrainingArguments(
    output_dir="gender_classification",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=1,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    logging_steps=logging_steps
)

In [27]:
#?Trainer

In [28]:
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"]
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: word_ids, text. If word_ids, text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 4845668
  Num Epochs = 1
  Instantaneous batch size per device = 280
  Total train batch size (w. parallel, distributed & accumulation) = 280
  Gradient Accumulation steps = 1
  Total optimization steps = 17306
